# 2. Quantizing embeddings

In [1]:
import os
import time
import torch
import pandas as pd
import numpy as np
from torch import nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, fbeta_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score, confusion_matrix
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
import random
import pickle
import sys

from torch.ao.quantization.observer import MinMaxObserver
# !pip install qnet

sys.path.insert(0, os.path.abspath('../'))
from qnet import *

In [2]:
class Args:
    # data_path = './efficientnet_b3_1536_bs64'
    # data_path = './efficientnet_b3_1536_bs64_PCA0.95'
    data_path = '/home/sebastian/codes/repo_clean/VE_paper/0_VE_extraction/E_NET_EMBEDDINGS/urban8k_efficientnet_b3/efficientnet_b3_16_bs64'
    output_dir = 'output'
    total_num_seed = 20
    num_classes = 5
    num_epochs = 500
    batch_size = 64
    learning_rate = 1e-3
    model = 'mlp'
    device = 'cpu'#'cuda' if torch.cuda.is_available() else 'cpu'
    pca = False
    variance_threshold = 0.8

args = Args()

# Fix paths to use current working directory
device = torch.device(args.device)
print(f"Using device: {device}")

Using device: cpu


In [3]:
df = pd.read_csv(os.path.join(args.data_path, 'val_embeddings.csv'))
features_tensor = torch.tensor(df.iloc[:, :-1].values, dtype=torch.float32)
observer = MinMaxObserver(dtype=torch.qint8, quant_min=-128, quant_max=127)
observer(features_tensor)
scale, zero_point = observer.calculate_qparams()
scale = scale.item()
zero_point = zero_point.item()
print(scale, zero_point, features_tensor.dtype)
quantized_tensor = torch.quantize_per_tensor(
            features_tensor, 
            scale, 
            int(zero_point), 
            dtype=torch.qint8
        )
print("quantized embedding: ", quantized_tensor.dtype)
quant_data = quantized_tensor.int_repr().numpy()
quant_data

0.004740460775792599 12 torch.float32
quantized embedding:  torch.qint8


array([[-13,  48,  13, ...,   7,  11,  12],
       [ -7,  64,   8, ...,   9, -15,  -3],
       [-21,  66,  -8, ...,  22, -20, -25],
       ...,
       [ 35,  59,  28, ...,  35, -49, -14],
       [ -2,  38,  12, ...,  20,  -3,   3],
       [ 65,  56,  68, ...,  -1,  78,  -2]], dtype=int8)

In [4]:
features_tensor

tensor([[-0.1179,  0.1692,  0.0062,  ..., -0.0226, -0.0058,  0.0018],
        [-0.0894,  0.2455, -0.0177,  ..., -0.0148, -0.1300, -0.0726],
        [-0.1555,  0.2572, -0.0927,  ...,  0.0473, -0.1511, -0.1742],
        ...,
        [ 0.1079,  0.2227,  0.0741,  ...,  0.1103, -0.2909, -0.1238],
        [-0.0650,  0.1220,  0.0020,  ...,  0.0397, -0.0712, -0.0445],
        [ 0.2531,  0.2099,  0.2653,  ..., -0.0626,  0.3118, -0.0678]])

In [5]:
#dequantize
quantized = quant_data.astype(np.float32)

In [6]:
dequantized = torch.dequantize(quantized_tensor)
dequantized

tensor([[-0.1185,  0.1707,  0.0047,  ..., -0.0237, -0.0047,  0.0000],
        [-0.0901,  0.2465, -0.0190,  ..., -0.0142, -0.1280, -0.0711],
        [-0.1564,  0.2560, -0.0948,  ...,  0.0474, -0.1517, -0.1754],
        ...,
        [ 0.1090,  0.2228,  0.0758,  ...,  0.1090, -0.2892, -0.1233],
        [-0.0664,  0.1233,  0.0000,  ...,  0.0379, -0.0711, -0.0427],
        [ 0.2512,  0.2086,  0.2655,  ..., -0.0616,  0.3129, -0.0664]])

In [7]:
dequantized.dtype

torch.float32

# Save quantized embeddings 

In [8]:
quantized_data = True  # Toggle data quantization

train_csv = os.path.join(args.data_path, 'train_embeddings.csv')
val_csv = os.path.join(args.data_path, 'val_embeddings.csv')
    
experiment_folder = f"{args.model}_{args.data_path.split('/')[-1]}"
if quantized_data:
        experiment_folder += "_data_quantized"
output_folder = os.path.join(os.getcwd(), args.output_dir, experiment_folder)
quantized_dir = os.path.join(output_folder, f"experiment_folder_quantizedVE")

os.makedirs(quantized_dir, exist_ok=True)
    
train_dataset = QuantizedEmbeddingDataset(train_csv, save_dir=quantized_dir, train=True, save_embeddings=True)
val_dataset = QuantizedEmbeddingDataset(val_csv, save_dir=quantized_dir, train=False, save_embeddings=True)

Quantized embeddings saved to /home/sebastian/codes/repo_clean/VE_paper/1_Classification/output/mlp_efficientnet_b3_16_bs64_data_quantized/experiment_folder_quantizedVE/quantized_embeddings_train.csv
Quantized embeddings saved to /home/sebastian/codes/repo_clean/VE_paper/1_Classification/output/mlp_efficientnet_b3_16_bs64_data_quantized/experiment_folder_quantizedVE/quantized_embeddings_val.csv
